In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
!ls

/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save
id26.0.txt  id32.0.txt	id6.0.txt


###Load data don't work in state diagram

In [8]:
for i in range(0,4):
  if i == 2:
    break
  print(i)

0
1


In [22]:
import os
import math
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

class MyCustomDataset(Sequence):
    def __init__(self, info_files, batch_size=1, max_len=147):
        """Instaniate a custom dataset.

        Attributes
          info_files (list or str): List of info files or only one file. Each file
            should have pattern like 'DendoX/*/info.txt' or 'DenxanhX/*/info.txt'.
          batch_size (int): Batch size.
          max_len (int): Maximum of length of sequence.
        """
        self.batch_size = batch_size
        self.max_len = max_len
        if isinstance(info_files, str):
            info_files = [info_files]
        
        for path in info_files:
            assert os.path.isfile(path), f'Not found info file: {path}'
        self.data = self._load_data(info_files)
    
    def on_epoch_end(self):
        np.random.shuffle(self.data)

    def _load_data(self, file_paths):
        data = []
        for path in file_paths:
            # Load label from file path.
           
            # Store object moving states in a list. So this should be a list of
            # 4-value list.
            sequences = []
            central_list = []
            bbox_list = []
            with open(path) as f:
                for line in f:
                    line = line.strip()
                    row = line.split(',')
                    #row = list(map(float, row))  # convert to float
                    bboxes = row[-4:]
                    sequences.append(bboxes)
                    bbox_list.append(bboxes)
            #process label
            name_province = path.split('/')[-2]
            check = 0
            if 'hanam' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 797:
                    check = 1
                elif float(bbox[3]) > 797:
                    check = 2
                    break
            if 'thainguyen' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 839:
                    check = 1
                elif float(bbox[3]) > 839:
                    check = 2
                    break
            if 'vungtau' in name_province.lower():
              for bbox in bbox_list:
                if float(bbox[3]) < 727:
                    check = 1
                elif float(bbox[3]) > 727:
                    check = 2
                    break
            if (check == 2): 
              label = 1 #hoạt động được với sơ đồ trạng thái
            if (check == 1):
              label = 0 #khong hoạt động được với sơ đồ trạng thái
            #padding 
            #sequences_pad = keras.preprocessing.sequence.pad_sequences([sequences], maxlen=147, padding="pre")
            while len(sequences) < 147:
              sequences.insert(0, [0,0,0,0])

            sequences_pad = sequences[:self.max_len]
            data.append((sequences_pad, label))
        return data
    
    def __len__(self):
        return math.ceil(len(self.data) / self.batch_size)
    
    def __getitem__(self, index):
        """Returns a batch of shape (N, S, 4)"""
        batch_data = self.data[index:self.batch_size:(index+1)*self.batch_size]
        X_batch = np.empty((self.batch_size, self.max_len, 4))
        y_batch = []
        for sequences, label in batch_data:
            X_batch[:len(sequences), :, :] =  np.array(sequences, dtype='float32')
            y_batch.append(label)
        X_batch = np.array(X_batch, dtype='float32')
        y_batch = np.array(y_batch, dtype='int32')
        return X_batch, y_batch

In [23]:
# sắp xếp tên thư mục giống trên window
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

###Thống kê trên tập dataset

In [24]:
path = '/content/drive/MyDrive/Project/dataset/dendo/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['hanam_14.0.avi_save',
 'hanam_20.0.avi_save',
 'hanam_23.0.avi_save',
 'hanam_26.0.avi_save',
 'hanam_46.0.avi_save',
 'hanam_52.0.avi_save',
 'hanam_55.0.avi_save',
 'hanam_72.0.avi_save',
 'hanam_75.0.avi_save',
 'hanam_81.0.avi_save',
 'hanam_101.0.avi_save',
 'thainguyen_0.0.avi_save',
 'thainguyen_3.0.avi_save',
 'thainguyen_4.0.avi_save',
 'thainguyen_7.0.avi_save',
 'thainguyen_8.0.avi_save',
 'thainguyen_11.0.avi_save',
 'thainguyen_12.0.avi_save',
 'thainguyen_15.0.avi_save',
 'thainguyen_18.0.avi_save',
 'thainguyen_19.0.avi_save',
 'thainguyen_22.0.avi_save',
 'thainguyen_23.0.avi_save',
 'thainguyen_25.0.avi_save',
 'thainguyen_26.0.avi_save',
 'thainguyen_30.0.avi_save',
 'thainguyen_33.0.avi_save',
 'thainguyen_34.0.avi_save',
 'thainguyen_36.0.avi_save',
 'thainguyen_37.0.avi_save',
 'thainguyen_40.0.avi_save',
 'thainguyen_41.0.avi_save',
 'thainguyen_44.0.avi_save',
 'thainguyen_45.0.avi_save',
 'thainguyen_51.0.avi_save',
 'thainguyen_52.0.avi_save',
 'thainguyen_55

In [25]:
dataset = []
label = []
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)
"""
info_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'
dataset = MyCustomDataset(info_files)
for batch in dataset:
    print(batch[0].shape, batch[1].shape)
    break

print(batch[0])
"""

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

"\ninfo_files = '/content/drive/MyDrive/Project/dataset/dendo/hanam_14.0.avi_save/id9.0.txt'\ndataset = MyCustomDataset(info_files)\nfor batch in dataset:\n    print(batch[0].shape, batch[1].shape)\n    break\n\nprint(batch[0])\n"

In [26]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)

In [27]:
print(data.shape)
print(label_data.shape)

(253, 147, 4)
(253,)


In [ ]:
data[0]

array([[ 681.875 , 1421.787 ],
       [ 683.281 , 1421.698 ],
       [ 686.875 , 1421.759 ],
       [ 681.875 , 1422.912 ],
       [ 683.75  , 1424.267 ],
       [ 678.281 , 1423.8676],
       [ 678.7525, 1422.3055],
       [ 677.031 , 1420.6245],
       [ 677.969 , 1420.948 ],
       [ 678.906 , 1420.0885],
       [ 678.75  , 1419.403 ],
       [ 678.281 , 1419.3375],
       [ 678.7475, 1419.08  ],
       [ 677.5   , 1418.6794],
       [ 677.5   , 1419.002 ],
       [ 677.4975, 1418.388 ],
       [ 677.4975, 1418.5615],
       [ 678.1275, 1418.486 ],
       [ 677.5   , 1418.426 ],
       [ 677.5025, 1418.436 ],
       [ 678.125 , 1418.741 ],
       [ 677.5   , 1418.3055],
       [ 677.5025, 1418.5145],
       [ 677.969 , 1418.793 ],
       [ 678.281 , 1418.6865],
       [ 677.344 , 1418.9865],
       [ 676.875 , 1418.0485],
       [ 682.656 , 1418.1045],
       [ 683.1275, 1417.883 ],
       [ 682.5   , 1417.7485],
       [ 682.656 , 1417.562 ],
       [ 683.1275, 1417.583 ],
       [

In [28]:
label

[array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], 

In [29]:
path = '/content/drive/MyDrive/Project/dataset/denxanh/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['thainguyen_0.0.avi_save',
 'thainguyen_3.0.avi_save',
 'thainguyen_4.0.avi_save',
 'thainguyen_7.0.avi_save',
 'thainguyen_8.0.avi_save',
 'thainguyen_11.0.avi_save',
 'thainguyen_12.0.avi_save',
 'thainguyen_14.0.avi_save',
 'thainguyen_15.0.avi_save',
 'thainguyen_18.0.avi_save',
 'thainguyen_19.0.avi_save',
 'thainguyen_22.0.avi_save',
 'thainguyen_23.0.avi_save',
 'thainguyen_25.0.avi_save',
 'thainguyen_26.0.avi_save',
 'thainguyen_30.0.avi_save',
 'thainguyen_33.0.avi_save',
 'thainguyen_34.0.avi_save',
 'thainguyen_36.0.avi_save',
 'thainguyen_37.0.avi_save',
 'thainguyen_40.0.avi_save',
 'thainguyen_41.0.avi_save',
 'thainguyen_44.0.avi_save',
 'thainguyen_45.0.avi_save',
 'thainguyen_51.0.avi_save',
 'thainguyen_52.0.avi_save',
 'thainguyen_55.0.avi_save',
 'thainguyen_56.0.avi_save',
 'thainguyen_59.0.avi_save',
 'thainguyen_62.0.avi_save',
 'thainguyen_63.0.avi_save',
 'thainguyen_66.0.avi_save',
 'thainguyen_67.0.avi_save',
 'vungtau_1.0.avi_save',
 'vungtau_12.0.avi_save

In [30]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [31]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)
print(data.shape)
print(label_data.shape)


(849, 147, 4)
(849,)


In [32]:
label

[array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([1], dtype=int32),
 array([0], dtype=int32),
 array([0], dtype=int32),
 array([1], dtype=int32),
 array([1], 

In [33]:
cnt_0 = 0
cnt_1 = 0
for i in label:
  if i == 0:
    cnt_0 += 1
  else:
    cnt_1 += 1
print(cnt_0)
print(cnt_1)

71
778


####Test

In [36]:
path = '/content/drive/MyDrive/Project/data_test/dendo/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['vungtau_59.0.avi_save',
 'vungtau_64.0.avi_save',
 'vungtau_65.0.avi_save',
 'vungtau_69.0.avi_save',
 'vungtau_70.0.avi_save',
 'vungtau_75.0.avi_save',
 'vungtau_76.0.avi_save',
 'vungtau_81.0.avi_save',
 'vungtau_82.0.avi_save',
 'vungtau_87.0.avi_save',
 'vungtau_92.0.avi_save',
 'vungtau_93.0.avi_save',
 'vungtau_98.0.avi_save',
 'vungtau_99.0.avi_save',
 'vungtau_103.0.avi_save',
 'vungtau_104.0.avi_save',
 'vungtau_109.0.avi_save',
 'vungtau_110.0.avi_save',
 'vungtau_120.0.avi_save',
 'vungtau_121.0.avi_save',
 'vungtau_126.0.avi_save',
 'vungtau_127.0.avi_save',
 'vungtau_132.0.avi_save',
 'vungtau_133.0.avi_save',
 'vungtau_137.0.avi_save',
 'vungtau_138.0.avi_save',
 'vungtau_143.0.avi_save',
 'vungtau_144.0.avi_save',
 'vungtau_149.0.avi_save',
 'vungtau_150.0.avi_save',
 'vungtau_161.0.avi_save',
 'vungtau_166.0.avi_save',
 'vungtau_167.0.avi_save',
 'vungtau_171.0.avi_save',
 'vungtau_172.0.avi_save',
 'vungtau_177.0.avi_save',
 'vungtau_178.0.avi_save',
 'vungtau_183.0

In [37]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [38]:
path = '/content/drive/MyDrive/Project/data_test/denxanh/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['hanam_1.0.avi_save',
 'hanam_4.0.avi_save',
 'hanam_30.0.avi_save',
 'hanam_33.0.avi_save',
 'hanam_59.0.avi_save',
 'hanam_62.0.avi_save',
 'hanam_88.0.avi_save',
 'hanam_91.0.avi_save']

In [39]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [40]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)
print(data.shape)
print(label_data.shape)

(1395, 147, 4)
(1395,)


####Test_new

In [41]:
path = '/content/drive/MyDrive/Project/data_test_new/denxanh/'
os.chdir(path)
os.listdir()
sort_listdir = sorted_alphanumeric(os.listdir())
sort_listdir

['vungtau_17.0.avi_save',
 'vungtau_18.0.avi_save',
 'vungtau_147.0.avi_save',
 'vungtau_152.0.avi_save',
 'vungtau_157.0.avi_save',
 'vungtau_163.0.avi_save',
 'vungtau_169.0.avi_save',
 'vungtau_174.0.avi_save',
 'vungtau_175.0.avi_save',
 'vungtau_181.0.avi_save',
 'vungtau_185.0.avi_save',
 'vungtau_186.0.avi_save',
 'vungtau_191.0.avi_save',
 'vungtau_196.0.avi_save']

In [42]:
for folder in sort_listdir:
  path_folder = path + folder + '/'
  os.chdir(path_folder)
  for file in sorted_alphanumeric(os.listdir()):
    info_files = path_folder + file
    dataset_temp = MyCustomDataset(info_files)
    for batch in dataset_temp:
      print(batch[0].shape, batch[1].shape)
      dataset.append(batch[0])
      label.append(batch[1])
  os.chdir(path)

(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1,)
(1, 147, 4) (1

In [43]:
data = np.concatenate(dataset, axis=0)
label_data = np.concatenate(label, axis=0)
print(data.shape)
print(label_data.shape)

(1618, 147, 4)
(1618,)


In [44]:
cnt_0 = 0
cnt_1 = 0
for i in label:
  if i == 0:
    cnt_0 += 1
  else:
    cnt_1 += 1
print(cnt_0)
print(cnt_1)

80
1538


####Ghi ra file csv

In [45]:
#ghi ra file data
import csv
b = open('/content/drive/MyDrive/Project/state_diagram_data.csv', 'w')
a = csv.writer(b)
a.writerows(data)
b.close()

In [46]:
#ghi ra file label
np.savetxt('/content/drive/MyDrive/Project/state_diagram_data_label.csv', label_data, delimiter=",", fmt='%d')